### Gráficos de barras

Plotly Express es una interfaz de alto nivel y fácil de usar para Plotly, que opera con una variedad de tipos de datos y produce figuras fáciles de diseñar .

Con px.bar, cada fila del DataFrame se representa como una marca rectangular. Para agregar múltiples puntos de datos en la misma marca rectangular, consulte la documentación del histograma.

En el siguiente ejemplo, solo hay una fila de datos por año, por lo que se muestra una sola barra por año.


In [1]:
!pip install dash #Opcional si no lo tienen instalado

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.3 MB/s eta 0:00:00


In [2]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_canada, x='year', y='pop')
fig.show()

##### Gráficos de barras con datos de formato largo
Los datos de formato largo tienen una fila por observación y una columna por variable. Esto es adecuado para almacenar y mostrar datos multivariados, es decir, con una dimensión mayor que 2. Este formato a veces se denomina "ordenado".


In [3]:
long_df = px.data.medals_long()

fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

In [4]:
long_df


,nation,medal,count
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15
5,Canada,silver,12
6,South Korea,bronze,11
7,China,bronze,8
8,Canada,bronze,12


##### Gráficos de barras con datos de formato ancho
Los datos de formato ancho tienen una fila por cada valor de una de las primeras variables y una columna por cada valor de la segunda variable. Esto resulta adecuado para almacenar y mostrar datos bidimensionales.

In [5]:
wide_df = px.data.medals_wide()

fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

In [6]:
wide_df

,nation,gold,silver,bronze
0,South Korea,24,13,11
1,China,10,15,8
2,Canada,9,12,12


##### Gráficos de barras en Dash
Dash es la mejor manera de crear aplicaciones analíticas en Python usando las figuras de Plotly. Para ejecutar la aplicación que aparece a continuación, ejecute pip install dash, haga clic en "Descargar" para obtener el código y ejecute python app.py.



In [8]:
app = Dash(__name__)
app.layout = html.Div([
    html.H4('Restaurant tips by day of week'),
    dcc.Dropdown(
        id="dropdown",
        options=["Fri", "Sat", "Sun"],
        value="Fri",
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"),
    Input("dropdown", "value"))
def update_bar_chart(day):
    df = px.data.tips() # replace with your own data source
    mask = df["day"] == day
    fig = px.bar(df[mask], x="sex", y="total_bill",
                 color="smoker", barmode="group")
    return fig


app.run_server(debug=True)

<IPython.core.display.Javascript object>

##### Barras de colores
El gráfico de barras se puede personalizar utilizando argumentos de palabras clave, por ejemplo, para utilizar un color continuo , como se muestra a continuación, o un color discreto , como se muestra arriba.

In [9]:

df = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(df, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig.show()

In [10]:

df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.bar(df, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='country',
             labels={'pop':'population of Canada'}, height=400)
fig.show()

##### Barras apiladas vs. barras agrupadas
Cuando varias filas comparten el mismo valor x(aquí Femenino o Masculino), los rectángulos se apilan uno sobre otro de forma predeterminada.

In [11]:
df = px.data.tips()
fig = px.bar(df, x="sex", y="total_bill", color='time')
fig.show()

El comportamiento predeterminado del gráfico de barras apiladas se puede cambiar a agrupado (también conocido como en clúster) usando el barmode argumento:

In [12]:
df = px.data.tips()
fig = px.bar(df, x="sex", y="total_bill",
             color='smoker', barmode='group',
             height=400)
fig.show()

##### Agregación en barras de un solo color
Como se indicó anteriormente, px.bar()se dibujará un rectángulo por fila de entrada . Esto a veces puede generar un aspecto rayado como en los ejemplos anteriores. Para combinar estos rectángulos en uno por color por posición, puede usar px.histogram(), que tiene su propia página de documentación detallada .

In [13]:
df = px.data.tips()
fig = px.histogram(df, x="sex", y="total_bill",
             color='smoker', barmode='group',
             height=400)
fig.show()

px.histogram() agregará y valores sumándolos de manera predeterminada, pero el histfunc argumento se puede usar para configurar esto para avg crear lo que a veces se denomina un "diagrama de barras", que resume la tendencia central de un conjunto de datos, en lugar de representar visualmente la totalidad del conjunto de datos.

In [14]:
df = px.data.tips()
fig = px.histogram(df, x="sex", y="total_bill",
             color='smoker', barmode='group',
             histfunc='avg',
             height=400)
fig.show()

##### Gráficos de barras con texto

Puede agregar texto a las barras mediante el text_auto argumento. Si lo configura como True se mostrarán los valores en las barras y, si lo configura como una d3-format cadena de formato, se controlará el formato de salida.

El text argumento se puede utilizar para mostrar texto en las barras:



In [15]:
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation", text_auto=True)
fig.show()

In [16]:
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation", text="nation")
fig.show()

##### Subplots facetados
Utilice la palabra clave arguments facet_row(resp. facet_col) para crear subparcelas facetadas, donde diferentes filas (resp. columnas) corresponden a diferentes valores de la columna del marco de datos especificada en facet_row.

In [17]:
df = px.data.tips()
fig = px.bar(df, x="sex", y="total_bill", color="smoker", barmode="group",
             facet_row="time", facet_col="day",
             category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
                              "time": ["Lunch", "Dinner"]})
fig.show()

##### Gráficos de barras básicos con plotly.graph_objects
Si Plotly Express no proporciona un buen punto de partida, también es posible utilizar la clase más genérica go.Bardeplotly.graph_objects .

In [18]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure([go.Bar(x=animals, y=[20, 14, 23])])
fig.show()

##### Gráfico de barras agrupadas
Personaliza la figura usando fig.update.

In [19]:
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Gráfico de barras apiladas


In [20]:
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

Gráfico de barras con texto flotante

In [21]:
x = ['Product A', 'Product B', 'Product C']
y = [20, 14, 23]

# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=x, y=y,
            hovertext=['27% market share', '24% market share', '19% market share'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='January 2013 Sales Report')
fig.show()

Personalización de colores de barras individuales


In [22]:

colors = ['lightslategray',] * 5
colors[1] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=['Feature A', 'Feature B', 'Feature C',
       'Feature D', 'Feature E'],
    y=[20, 14, 23, 25, 22],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Least Used Feature')

Personalización de anchos de barras individuales


In [23]:

fig = go.Figure(data=[go.Bar(
    x=[1, 2, 3, 5.5, 10],
    y=[10, 8, 6, 4, 2],
    width=[0.8, 0.8, 0.8, 3.5, 4] # customize width here
)])

fig.show()

Barras redondeadas


In [27]:
import plotly.graph_objects as go
from plotly import data

df = data.medals_wide()

fig = go.Figure(
    data=[
        go.Bar(x=df.nation, y=df.gold, name="Gold"),
        go.Bar(x=df.nation, y=df.silver, name="Silver"),
        go.Bar(x=df.nation, y=df.bronze, name="Bronze"),
    ],
    # layout=dict(
    #     barcornerradius=15,
    # ),
)

fig.show()

Gráficos de barras con eje de categorías múltiples


In [28]:

x = [
    ["BB+", "BB+", "BB+", "BB", "BB", "BB"],
    [16, 17, 18, 16, 17, 18,]
]
fig = go.Figure()
fig.add_bar(x=x,y=[1,2,3,4,5,6])
fig.add_bar(x=x,y=[6,5,4,3,2,1])
fig.update_layout(barmode="relative")
fig.show()